Importing necessary packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Plotting the energy spectrum of the Harper-Hofstadter Hamiltonian (real space)

In [ ]:
N = 20
t_nn, t_nnn = 1, 0.2
alphas = np.arange(0, N**2+1, 1) / N**2
energies_nn = []
alpha_vals = []

for alpha in alphas:
    eigvals = H_real(N, alpha, t_nn, t_nnn)
    energies.append(eigvals)
    alpha_vals.append([alpha]*len(eigvals))

fig = plt.figure(figsize=(10, 8))
ax = plt.axes()
ax.set_xlabel(r"$\alpha$", va = "center", size = "large", labelpad=10.0)
ax.set_ylabel(r"$\epsilon$", va = "center", rotation = "horizontal", size = "large", labelpad = 10.0)

for i in range(len(alpha_vals)):
    plt.scatter(alpha_vals[i], energies_nn[i], s = 0.00001, color='k')

Plotting the energy spectrum in k-space

In [ ]:
q = 3 # Denominator of alpha
t_nn, t_nnn = 1, 0.2

N = 20*q      
k_x = np.arange(-N//2,N//2)*(2*np.pi/N)
k_y = np.arange(-N//2,N//2)*(2*np.pi/(q*N))
K_x, K_y = np.meshgrid(k_x, k_y)

energies = np.zeros((len(k_x), len(k_y), q))
for i, kx_val in enumerate(k_x):
    for j, ky_val in enumerate(k_y):
        eigenvalues, eigenvectors = H_k(kx_val, ky_val, q, t_nn, t_nnn)
        energies[i, j] = np.sort(eigenvalues)

fig = plt.figure(figsize=(8,8))
ax = plt.axes(projection ='3d')
for i in range(q):
    ax.plot_surface(K_x/np.pi, K_y/np.pi, energies[:, :, i], color = 'grey')

ax.set_xlabel(r"$k_x$ $(\pi/a)$")
ax.set_ylabel(r"$k_y$ $(\pi/a)$")
ax.set_zlabel("$\epsilon$")
ax.view_init(azim = 0, elev = 0)

Plotting the quasienergies along a path through high symmetry points in the Brillouin Zone

In [ ]:
# Plotting the quasienergies by unfolding the magnetic Brillioun zone, moving from the gamma point (0, 0), to the Y point (0, pi/(a*q)), 
# to the S point (pi/a, pi/(a*q)), and back to Gamma point

q_1, q_2 = 2, 3 # Numerator and denominator of alpha
t_nn, t_nnn = 1, 0.2
T_1, T_2 = 0.3, 0.7

N = 20*q      
k_x = np.arange(-N//2,N//2)*(2*np.pi/N)
k_y = np.arange(-N//2,N//2)*(2*np.pi/(q*N))

quasienergies, eigenvectors = H_F_exact(0, 0, q_1, q_2, q, T_1, T_2)
sorted_energies = [quasienergies] #Initializing an array of energies to be plotted

k_x_pos = np.sort(k_x[k_x > 0])
k_y_pos = np.sort(k_y[k_y > 0])
k_return = [(k_x_pos[-i], k_y_pos[-i]) for i in range(2, len(k_x_pos) + 1)]

for ky_val in k_y_pos:
    quasienergies2, eigenvectors2 = H_F_exact(0, ky_val, q_1, q_2, q, T_1, T_2)
    overlap_mat = np.abs(np.transpose(np.conj(eigenvectors)) @ eigenvectors2) # overlap matrix
    row_ind, col_ind = linear_sum_assignment(-overlap_mat) # maximizing the overlap between neighboring states
    sorted_energies.append(quasienergies2[col_ind])
    eigenvectors = eigenvectors2[:, col_ind]

for kx_val in k_x_pos:
    quasienergies2, eigenvectors2 = H_F_exact(kx_val, ky_val, q_1, q_2, q, T_1, T_2)
    overlap_mat = np.abs(np.transpose(np.conj(eigenvectors)) @ eigenvectors2) # overlap matrix
    row_ind, col_ind = linear_sum_assignment(-overlap_mat) # maximizing the overlap between neighboring states
    sorted_energies.append(quasienergies2[col_ind])
    eigenvectors = eigenvectors2[:, col_ind]
    
for k_point in k_return:
    quasienergies2, eigenvectors2 = H_F_exact(k_point[0], k_point[1], q_1, q_2, q, T_1, T_2)
    overlap_mat = np.abs(np.transpose(np.conj(eigenvectors)) @ eigenvectors2) # overlap matrix
    row_ind, col_ind = linear_sum_assignment(-overlap_mat) # maximizing the overlap between neighboring states
    sorted_energies.append(quasienergies2[col_ind])
    eigenvectors = eigenvectors2[:, col_ind]   

for i in range(q):
    plt.scatter(range(len(np.array(sorted_energies)[:, i])), np.array(sorted_energies)[:, i], s=0.2)

plt.xticks([0, len(k_y_pos), len(k_y_pos) + len(k_x_pos), len(sorted_energies)], [r"$\Gamma$", "Y", "S", r"$\Gamma$"])
plt.ylabel(r"$\epsilon$", va = "center", rotation = "horizontal", size = "large", labelpad = 10.0)

Plotting a Floquet-Hofstadter Butterfly for alpha, -alpha switching

In [ ]:
T_1, T_2 = 0.3, 0.7
t_nn, t_nnn = 1, 0.2

chosen_alphas = np.array([[0, 0]])
q_vals = np.arange(30, 1, -1)
for q_val in q_vals:
    for p_val in range(1, q_val+1):
        if p_val/q_val in list(chosen_alphas[:, 0]):
            continue
        else:
            chosen_alphas = np.append(chosen_alphas, [[p_val/q_val, q_val]], axis=0)

energies = []
alpha_vals = []
for j in range(len(chosen_alphas)):
    if chosen_alphas[j, 1] < 8:
        N_val = 3*int(chosen_alphas[j, 1])
    elif chosen_alphas[j, 1] < 15:
        N_val = 2*int(chosen_alphas[j, 1])
    else:
        N_val = int(chosen_alphas[j, 1])
        
    Phi_val = 2*np.pi*chosen_alphas[j, 0]
    quasienergies = H_F_real(N_val, Phi_val, -Phi_val, T_1, T_2, t_nn, t_nnn)
    energies.append(quasienergies)
    alpha_vals.append([chosen_alphas[j, 0]]*len(quasienergies))

fig = plt.figure(figsize=(10,8))
ax = plt.axes()
for i in range(len(energies)):  
    plt.scatter(alphas[i], energies[i], marker='.', color='k', s=0.2)


ax.set_ylabel(r"$\epsilon$", rotation='horizontal', va='center', size='large', labelpad=10.0)
ax.set_xlabel(r"$\alpha$", va='center', size='large', labelpad=10.0)

Plotting the quasienergies in real space for alpha_1, alpha_2 switching as a function of T_2/T with T_1 = T - T_2

In [ ]:
q_1, q_2 = 3, 5
N = 2*np.lcm(q_1, q_2)
t_nn, t_nnn = 1, 0.2

T_max = 1
r = np.linspace(0, T_max, 100)
energies = []
r_vals = []
for r_val in r:
    T_1 = T_max - r_val
    T_2 = r_val
    quasienergies = H_F_real(N, q_1, q_2, T_1, T_2, t_nn, t_nnn)
    energies.append(quasienergies)
    r_vals.append([r_val]*len(quasienergies))
for i in range(len(energies)):
    plt.scatter(r_vals[i], energies[i], marker='.', color='k', s=0.05)

plt.xlabel(r"$T_2/T$", size=12)
plt.ylabel(r"$\epsilon$", size=14, rotation='horizontal', va="center", labelpad=10)

Plotting quasienergies for a cylindrical strip geometry supporting chiral edge modes

In [ ]:
q_1, q_2 = 3, 5
N = 20*np.lcm(q_1, q_2)
t_nn, t_nnn = 1, 0.2

energies = np.zeros((len(k_x), N))
for i, kx_val in enumerate(k_x):
    eigenvalues, eigenvectors, H = H_F_k_chiral(N, kx_val, q_1, q_2, T_1, T_2, t_nn, t_nnn)
    energies[i] = np.sort(eigenvalues)

fig = plt.figure(figsize=(8, 6))
for n in range(N):
    plt.plot(k_x/np.pi, energies[:, n], color='grey')
plt.xlabel(r"$k_x$ ($\pi/a$)", size=12)
plt.ylabel(r"$\epsilon$", rotation='horizontal', size=14, va='center')